# Possible thesis topic: Case study of a blocking in terms of EDA
## TODO:
- select case: 01-05-2017 06:00 UTC - 14-05-2017 00:00 UTC North Atlantic
- plot Var(EDA) gh500 field
- prepare syn_cond plot
    - obtain relevant ERA5 data
- investigate other fields
- check out uncertainty plots on ecmwf page (https://intraplots.ecmwf.int/packages/diagnostics/)

In [14]:
source("../RScripts/config.r")
source("../RScripts/plot_functions.r")
source("../RScripts/data_functions.r")
source("../RScripts/algo_functions.r")

options(repr.plot.width = 15, repr.plot.height = 10) # Adjust width & height


Linking to GEOS 3.11.1, GDAL 3.6.2, PROJ 9.1.1; sf_use_s2() is TRUE

Lade nötiges Paket: maps


Attache Paket: ‘rnaturalearthdata’


Das folgende Objekt ist maskiert ‘package:rnaturalearth’:

    countries110



Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union



Attache Paket: ‘purrr’


Das folgende Objekt ist maskiert ‘package:maps’:

    map



Attache Paket: ‘lubridate’


Die folgenden Objekte sind maskiert von ‘package:base’:

    date, intersect, setdiff, union


Lade nötiges Paket: zoo


Attache Paket: ‘zoo’


Die folgenden Objekte sind maskiert von ‘package:base’:

    as.Date, as.Date.numeric


Successfully loaded changepoint package version 2.3
 See NEWS for details of all changes.



In [15]:
data_path <- "/net/scratch/schoelleh96/WP2/WP2.2a/ens_data/"
file_name <- "spread_2017.nc"
library(ncdf4)


In [16]:
nc <- nc_open("/net/scratch/schoelleh96/WP2/WP2.2a/ens_data/spread_2017.nc")
z_data <- ncvar_get(nc, "z")

lat <- ncvar_get(nc, "latitude")
lon <- ncvar_get(nc, "longitude")
time_data <- ncvar_get(nc, "valid_time")
time_origin <- sub(
    "seconds since ", "",
    ncatt_get(nc, "valid_time", "units")$value
)
time <- as.Date(as.POSIXct(time_data,
    origin = time_origin,
    tz = "UTC"
))


In [18]:
# Define start and end dates
start_date <- as.Date("2017-05-01")
end_date <- as.Date("2017-05-15")

# Find indices corresponding to the desired time period
time_indices <- which(time >= start_date & time <= end_date)

# Loop through each time step in the subset
for (i in time_indices) {
    # Extract z data for the current time step (assuming z_data dimensions: [lon, lat, time])
    z_slice <- z_data[, , i]

    # Create a data frame with lon, lat, and the z values for this time step
    grid_df <- expand.grid(lon = lon, lat = lat)
    grid_df$log_z <- as.vector(log(z_slice))

    # Generate a plot for the current time step using your plot_spatial function
    p <- plot_spatial(
        data = grid_df,
        var_name = "log_z",
        legend_name = "log_z",
        title = paste("log(var(gh500)) on", time[i])
    )

    # Create a filename for the current plot (e.g., "z_20170501.pdf")
    filename <- paste0("../figs/z_", format(time[i], "%Y%m%d"), ".pdf")

    # Save the plot using the provided save_plot function
    save_plot(p, filename)
}
